In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim import corpora
from gensim import models
from gensim.models import CoherenceModel
from langid import set_languages, classify
set_languages(['nl', 'en'])

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

df_grote_word_doc = pd.read_json(r'C:\Users\johan\Documents\GitHub\sandbox\NLP\data\NLP for grote word-filev2.json')

# stopwords

In [ ]:
from nltk.corpus import stopwords

stop_words = stopwords.words('dutch') + stopwords.words('english') 

more_words = ['vooral', 'gaan', 'één', 'value', 'part', 'use', 'blijven', 'waarbij', 'stuk', 'wanneer', 'much', 'kennen', 'always', 'tegelijk', 'however', 'geven', 'nooit', 'weg', 'vaak', 'soort', 'wellicht', 'leggen', 'steken', 'leven', 'zoal,', 'waar', 'allemaal', 'net', 'eigen', 'stefaf', 'vallen', 'zaak', 'feit', 'waaruit', 'zelfs', 'year', 'echter', 'zien', 'come', 'willen', 'spreken', 'straf', 'lijken', 'staan', 'even', 'hoog', 'pas', 'liggen', 'waarom', 'helemaal', 'situatie', 'waaraan', 'zitten', 'take', 'waarin', 'often', 'wel', 'maken', 'nieuw', 'waarop', 'plots', 'say', 'goed', 'way', 'terug', 'mogelijk', 'many', 'daarom', 'omwille', 'leren', 'nemen', 'kijken', 'waarde', 'gebruiken', 'iphone', 'eerder', 'weer', 'zoeken', 'dienen', 'alleen', 'houden', 'see', 'well', 'good', 'deel', 'find', 'misschien', 'make', 'vinden', 'also', 'manier', 'natuurlijk', 'laten', 'louter', 'komen', 'stellen', 'ergens', 'live', 'ver', 'daarentegen', 'facebook', 'steeds', 'time', 'need', 'enkel', 'new', 'nodig', 'vormen', 'halen', 'duidelijk', 'zeggen', 'camera', 'krijgen', 'brengen', 'eigenlijk', 'proberen', 'gewoon', 'heel', 'zeer', 'telkens', 'look', 'eerst', 'belangrijk', 'nochtans', 'waarmee', 'lang', 'zeker']
more_words = more_words
stop_words = list(set(stop_words + more_words))
len(stop_words)

In [ ]:
def stopwords_preprocess(stop_words):
    for stop in stop_words:
        sent = gensim.utils.simple_preprocess(str(stop.strip()), deacc=True)
        if sent:
            yield(sent)
            
stop_words += ['zoals']
stop_words = [' '.join(w) for w in list(stopwords_preprocess(stop_words))]

'en' in stop_words

# preprocessing

In [ ]:
data = df_grote_word_doc.text.values.tolist()

def sent_to_words(texts):
    for text in texts:
        sent = gensim.utils.simple_preprocess(str(text), deacc=True, min_len=3) 
        yield' '.join(sent)

# Convert to list
data_words = list(sent_to_words(data))
len(data_words)

In [ ]:
# old
sentences = df_grote_word_doc.text.values.tolist()

def sent_to_wordsv2(sentences):
    for sent in sentences:
        sent_out = []
        sent = re.sub(r'\d+', ' ', sent)
        sent = re.sub(r'\W+', ' ', sent)
        for word in sent.split(' '):
            word = word.lower().strip()
            if word and word not in stop_words:
                sent_out.append(word) 
        yield ' '.join(sent_out)
        
# clean_word_list = list(sent_to_wordsv2(sentences))

len(clean_word_list)

# tf-idf

see digital humanities notebook. Based on https://www.youtube.com/watch?v=i74DVqMsRWY

In [ ]:
vectorizer = TfidfVectorizer(lowercase=True,
                            max_features=75, # tutorial put 100. when increasing it, there's a whole bunch on unclustered stuff in the middle
                            max_df=0.65, #  This ignores terms that appear in more than 60% of the documents.
                            min_df=2, #  This ignores terms that appear in fewer than 3 documents to filter out typos or extremely rare words. 
                            ngram_range = (1,3),
                            stop_words = stop_words,
                            # sublinear_tf=True
                            )

vectors = vectorizer.fit_transform(data_words) # 

feature_names = vectorizer.get_feature_names_out()
denselist = vectors.todense().tolist()

all_keywords = []
for description in denselist:
    x=0
    keywords = []
    for word in description:
        if word > 0:
            keywords.append(feature_names[x])
        x+=1
    all_keywords.append(keywords)
    
# print(data_words[0])
# print(all_keywords[0])

In [ ]:
true_k = 35

model = KMeans(n_clusters=true_k, init="k-means++", max_iter=400, n_init=10)

model.fit(vectors)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
for i in range(true_k):
    print(f"Cluster {i}")
    for ind in order_centroids[i, :5]:
        print('- ', feature_names[ind])

In [ ]:
# Retrieve the labels assigned by KMeans and organise them as a list of the same length of the emails
labels = model.labels_
keyword_list = []
for label in labels:    
    keywords_for_label = ', '.join(feature_names[i] for i in order_centroids[label, :5])
    keyword_list.append(keywords_for_label)

`colors = np.array(mycolors)[labels % len(mycolors)] # note that this kind of indexing only works on np.arrays not on lists`

In [ ]:
# Perform t-SNE dimensionality reduction
tsne_model = TSNE(n_components=2, verbose=0, random_state=20, angle=.99, init='pca', perplexity=95) #, learning_rate=50, early_exaggeration=35)
low_dim_data = tsne_model.fit_transform(np.array(vectors.todense()))

In [ ]:
from bokeh.models import ColumnDataSource, HoverTool, TapTool, CustomJS, Div
from bokeh.layouts import column

output_notebook()
# mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
mycolors = np.array(list(mcolors.TABLEAU_COLORS.values()) + \
         [mcolors.to_hex(c) for c in plt.cm.Pastel1.colors] + \
         [mcolors.to_hex(c) for c in plt.cm.Set1.colors])
         
source = ColumnDataSource(data=dict(
    x=low_dim_data[:, 0],
    y=low_dim_data[:, 1],
    colors=mycolors[labels % len(mycolors)],
    onderwerp=df_grote_word_doc.heading.tolist(),
    description=sentences,
    keywords = keyword_list))

hover = HoverTool()
hover.tooltips = [("onderwerp", "@onderwerp"),
                  ("keywords", "@keywords")
                  ]

description_div = Div(text="", width=1200, height=150)

callback = CustomJS(args=dict(source=source, div=description_div), code="""
    const indices = source.selected.indices;
    if (indices.length == 0)
        return;
    const desc = source.data['description'][indices[0]];
    div.text = desc;
""")
tap_tool = TapTool(callback=callback)

plot = figure(tools="wheel_zoom, reset", title=f"t-SNE Clustering of {true_k} KMeans Topics", width=1200, height=800)
plot.add_tools(hover)
plot.add_tools(tap_tool)

plot.scatter('x', 'y', source=source, color='colors')

layout = column(plot, description_div)

show(layout)

# Evaluation methods

With tf-idf => KMEANS => TSNE there are 3 models that need parameter tuning.

For TSNE I found it important to tweak perplexity and early exaggeration. see https://distill.pub/2016/misread-tsne/ and https://towardsdatascience.com/t-sne-clearly-explained-d84c537f53a

## K-MEANS

https://scikit-learn.org/stable/modules/clustering.html#k-means

- The K-means algorithm aims to choose centroids that minimise the inertia
- prone to Curse of Dimensionality > Running a dimensionality reduction algorithm such as Principal component analysis (PCA) prior to k-means clustering can alleviate this problem and speed up the computations.

In [ ]:
limit = 20

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

wcss = []
for i in range(2, limit):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=100, n_init=1)
    kmeans.fit(vectors)
    wcss.append(kmeans.inertia_) # Inertia measures the sum of squared distances of samples to their closest cluster center. A lower inertia indicates that the clusters are more compact.
    
plt.plot(range(2, limit), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

see here https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

The silhouette_score is a comparison between its tightness and separation

In [ ]:
from sklearn.metrics import silhouette_score

sil = []
for k in range(2, limit):
    kmeans = KMeans(n_clusters=k, n_init=1).fit(vectors)
    preds = kmeans.predict(vectors)
    sil.append(silhouette_score(vectors, preds, metric='euclidean'))

plt.plot(range(2, limit), sil)
plt.title('Silhouette Method')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.xticks()
plt.show()

In [ ]:
# from tutorial

from sklearn import metrics
from sklearn import datasets

X, y = datasets.load_iris(return_X_y=True)

import numpy as np
from sklearn.cluster import KMeans

kmeans_model = KMeans(n_clusters=3, random_state=1).fit(X)
labels = kmeans_model.labels_
metrics.silhouette_score(X, labels, metric='euclidean')

In [ ]:
metrics.davies_bouldin_score(X, labels)

more on DBI here https://scikit-learn.org/stable/modules/clustering.html#davies-bouldin-index

Values closer to zero indicate a better partition.

In [ ]:
from sklearn.metrics import davies_bouldin_score

db = []
for k in range(2, limit):
    kmeans = KMeans(n_clusters=k, n_init=1).fit(vectors)
    preds = kmeans.predict(vectors)
    db.append(davies_bouldin_score(vectors.toarray(), preds))

plt.plot(range(2, limit), db)
plt.title('Davies-Bouldin Index')
plt.xlabel('Number of clusters')
plt.ylabel('DBI')
plt.show()

In [ ]:
db[7]